In [1]:
import json
import pandas as pd
from tqdm import tqdm
from mibi.modules import DocumentsModule, Question
from test_utils import (
    get_snippets,
    PubMedApiRetrieve,
    rerank_biencoder,
    rerank_crossencoder,
    get_snippets,
    get_offset,
    response_exact_answer,
    response_ideal_answer,
    flat_list,
    get_snippets_blablador,
)

In [2]:
TEST_FILE = "../../data/bioasq-data/training12b_new.json"

with open(TEST_FILE, "r") as f:
    data = json.load(f)

data = data["questions"][:5]  # for the test purpose, take one instance
print(data[0].keys())

dict_keys(['body', 'documents', 'ideal_answer', 'concepts', 'type', 'id', 'snippets'])


In [3]:
RETRIEVE_TOP_K = 50
RERANK_TOP_K = 10
ABSTRACTS_FOR_ANSWER = 3

In [4]:
N = 0
questions = [
    Question(
        id=q["id"],
        type=q["type"],
        body=q["body"],
    )
    for q in data
]
for question in tqdm(questions):
    N += 1
    retrieve_abstracts = PubMedApiRetrieve(verbose=True, num_results=RETRIEVE_TOP_K)
    retrieved = retrieve_abstracts.transform([question])
    retrieved["cos_sim"] = rerank_crossencoder(question, retrieved)
    reranked = retrieved.sort_values("cos_sim", ascending=False)[:RERANK_TOP_K]
    # reranked["snippets_blablador_gpt"] = reranked.apply(
    #     lambda x: get_snippets_blablador(
    #         question.body, x["title"], x["text"], model="gpt-3.5-turbo"
    #     ),
    #     axis=1,
    # )
    # reranked["snippets_blablador_mistral"] = reranked.apply(
    #     lambda x: get_snippets_blablador(
    #         question.body, x["title"], x["text"], model="Mistral-7B-Instruct-v0.2"
    #     ),
    #     axis=1,
    # )
    # reranked["title_snippets_gpt"] = [x[0] for x in reranked["snippets_blablador_gpt"]]
    # reranked["abstract_snippets_gpt"] = [
    #     x[1] for x in reranked["snippets_blablador_gpt"]
    # ]
    # reranked["title_snippets_mistral"] = [
    #     x[0] for x in reranked["snippets_blablador_mistral"]
    # ]
    # reranked["abstract_snippets_mistral"] = [
    #     x[1] for x in reranked["snippets_blablador_mistral"]
    # ]

    # reranked["offset_title_gpt"] = reranked.apply(
    #     lambda x: get_offset(x["title_snippets_gpt"], x["text"]), axis=1
    # )
    # reranked["offset_abstract_gpt"] = reranked.apply(
    #     lambda x: get_offset(x["abstract_snippets_gpt"], x["text"]), axis=1
    # )

    # reranked["offset_title_mistral"] = reranked.apply(
    #     lambda x: get_offset(x["title_snippets_mistral"], x["text"]), axis=1
    # )
    # reranked["offset_abstract_mistral"] = reranked.apply(
    #     lambda x: get_offset(x["abstract_snippets_mistral"], x["text"]), axis=1
    # )

    # reranked["snippets"] = reranked.apply(
    #     lambda x: get_snippets(question.body, x["title"], x["text"]), axis=1
    # )
    # reranked["title_snippets"] = [x[0] for x in reranked["snippets"]]
    # reranked["abstract_snippets"] = [x[1] for x in reranked["snippets"]]
    # reranked["offset_title"] = reranked.apply(
    #     lambda x: get_offset(x["title_snippets"], x["text"]), axis=1
    # )
    # reranked["offset_abstract"] = reranked.apply(
    #     lambda x: get_offset(x["abstract_snippets"], x["text"]), axis=1
    # )

    # reranked["answer_abstracts_exact"] = [
    #     response_exact_answer(
    #         question.body,
    #         question.type,
    #         " ".join(reranked["text"].tolist()[:ABSTRACTS_FOR_ANSWER]),
    #     )
    # ] * len(reranked)
    # reranked["answer_abstracts_ideal"] = [
    #     response_ideal_answer(
    #         question.body,
    #         question.type,
    #         " ".join(reranked["text"].tolist()[:ABSTRACTS_FOR_ANSWER]),
    #     )
    # ] * len(reranked)
    # reranked["answer_snippets_exact"] = [
    #     response_exact_answer(
    #         question.body,
    #         question.type,
    #         " ".join(
    #             flat_list(reranked["abstract_snippets"].tolist())
    #             + flat_list(reranked["title_snippets"])
    #         ),
    #     )
    # ] * len(reranked)
    # reranked["answer_snippets_ideal"] = [
    #     response_ideal_answer(
    #         question.body,
    #         question.type,
    #         " ".join(
    #             flat_list(reranked["abstract_snippets"].tolist())
    #             + flat_list(reranked["title_snippets"])
    #         ),
    #     )
    # ] * len(reranked)

    reranked.to_csv(f"reranked{N}.csv", index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

Is Hirschsprung disease a mendelian or a multifactorial disorder?


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 60%|██████    | 3/5 [00:27<00:14,  7.19s/it]

Are long non coding RNAs spliced?


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 80%|████████  | 4/5 [00:32<00:06,  6.35s/it]

Is RANKL secreted from the cells?


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 5/5 [00:37<00:00,  7.49s/it]


In [5]:
questions[0].type, questions[0].body

('summary',
 'Is Hirschsprung disease a mendelian or a multifactorial disorder?')

In [16]:
reranked.head(15)

,docno,score,rank,title,text,url,cos_sim,snippets_blablador_gpt
55,38218304,0,56,Runx2 deletion in hypertrophic chondrocytes im...,Deletion of Runx2 gene in proliferating chondr...,https://pubmed.ncbi.nlm.nih.gov/38218304/,0.270721,"([], [])"
43,38266238,0,44,Myeloid zinc finger 1 knockdown promotes osteo...,The overactivation of the osteoclasts is a cru...,https://pubmed.ncbi.nlm.nih.gov/38266238/,0.234311,"([], [])"
25,38365111,0,26,FTO regulates osteoclast development by modula...,Periodontitis is an oral inflammatory disease ...,https://pubmed.ncbi.nlm.nih.gov/38365111/,0.232768,"([], [])"
10,38457001,0,11,Osteocytes and Paget's Disease of Bone.,To describe the contributions of osteocytes to...,https://pubmed.ncbi.nlm.nih.gov/38457001/,0.226861,"([], [])"
18,38396794,0,19,The Role of Rosavin in the Pathophysiology of ...,"Rosavin, a phenylpropanoid in",https://pubmed.ncbi.nlm.nih.gov/38396794/,0.218131,"([], [])"
30,38331308,0,31,A pathogenic role for brain-derived neurotroph...,Brain derived neurotrophic factor (BDNF) is a ...,https://pubmed.ncbi.nlm.nih.gov/38331308/,0.209336,"([], [])"
91,38048727,0,92,Dysfunction of poly (ADP-ribose) glycohydrolas...,Poly (ADP-ribose) glycohydrolase (PARG) is an ...,https://pubmed.ncbi.nlm.nih.gov/38048727/,0.196737,"([], [])"
22,38378678,0,23,RANKL inhibition reduces lesional cellularity ...,"Fibrous dysplasia (FD) is a rare, disabling sk...",https://pubmed.ncbi.nlm.nih.gov/38378678/,0.193740,"([], [])"
98,38003715,0,99,Ethanol Extract of Radix Asteris Suppresses Os...,"Radix Asteris, the root of",https://pubmed.ncbi.nlm.nih.gov/38003715/,0.184388,"([], [])"
84,38068903,0,85,Impact of Mechanical Strain and Nicotinamide o...,Cleidocranial dysplasia (CCD) is a rare geneti...,https://pubmed.ncbi.nlm.nih.gov/38068903/,0.162072,"([], [])"


In [9]:
print(reranked.title.tolist()[0])
print(reranked.text.tolist()[0])
print(reranked.snippets_blablador_gpt.tolist()[0])
print(reranked.snippets_blablador_mistral.tolist()[0])

Secreted ADAMTS-like proteins as regulators of connective tissue function.
The extracellular matrix (ECM) determines functional properties of connective tissues through structural components, such as collagens, elastic fibers, or proteoglycans. The ECM also instructs cell behavior through regulatory proteins, including proteases, growth factors, and matricellular proteins, which can be soluble or tethered to ECM scaffolds. The secreted a disintegrin and metalloproteinase with thrombospondin type 1 repeats/motifs-like (ADAMTSL) proteins constitute a family of regulatory ECM proteins that are related to ADAMTS proteases but lack their protease domains. In mammals, the ADAMTSL protein family comprises seven members, ADAMTSL1-6 and papilin. ADAMTSL orthologs are also present in the worm,
([], ['ADAMTSL orthologs are also present in the worm,', 'ADAMTSL proteins constitute a family of regulatory ECM proteins that are related to ADAMTS proteases but lack their protease domains.'])
(['Secrete

In [31]:
reranked.abstract_snippets.tolist()[4]

['The differentially expressed genes were mainly associated with catalytic activity, oxidoreductase activity and transmembrane transporter activity, which significantly contributed to extracellular matrix-receptor interaction, fatty acids biosynthesis as well as glycine, serine, and threonine metabolism.']

In [2]:
question = "Please list 2 human diseases caused by a coronavirus."
title = "Host Factors in Coronavirus Replication"
abstract = "Coronaviruses are pathogens with a serious impact on human and animal health. They mostly cause enteric or respiratory disease, which can be severe and life threatening, e.g., in the case of the zoonotic coronaviruses causing severe acute respiratory syndrome (SARS) and Middle East Respiratory Syndrome (MERS) in humans. Despite the economic and societal impact of such coronavirus infections, and the likelihood of future outbreaks of additional pathogenic coronaviruses, our options to prevent or treat coronavirus infections remain very limited. This highlights the importance of advancing our knowledge on the replication of these viruses and their interactions with the host. Compared to other +RNA viruses, coronaviruses have an exceptionally large genome and employ a complex genome expression strategy. Next to a role in basic virus replication or virus assembly, many of the coronavirus proteins expressed in the infected cell contribute to the coronavirus-host interplay. For example, by interacting with the host cell to create an optimal environment for coronavirus replication, by altering host gene expression or by counteracting the host's antiviral defenses. These coronavirus-host interactions are key to viral pathogenesis and will ultimately determine the outcome of infection. Due to the complexity of the coronavirus proteome and replication cycle, our knowledge of host factors involved in coronavirus replication is still in an early stage compared to what is known for some other +RNA viruses. This review summarizes our current understanding of coronavirus-host interactions at the level of the infected cell, with special attention for the assembly and function of the viral RNA-synthesising machinery and the evasion of cellular innate immune responses."

In [4]:
import re

a = "Lorem ipsum dolor sit amet"
re.search("ipsum", a).span()
(6, 11)

(6, 11)